# Cuaderno 7: Densidad Espectral de Potencia

Podemos ver cualquier señal digitan el el dominio **temporal** o **frecuencial**.

La Densidad Espectral de Potencia (PSD, por sus siglas en inglés) de una señal es una función matemática que nos informa de cómo está distribuida la potencia de dicha señal sobre las distintas frecuencias de las que está formada. Es una forma de visualizar que frecuencias están presentes en una señal.

En este cuaderno, veremos que es un espectro de potencia, y una forma simple de calcularla.

Para empezar, como es usual, cargamos las librerias que vamos a utilizar:

In [ ]:
import requests, io
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt

Y luego cargamos datos registrados en una electrocorticografía (ECoG):

In [ ]:
url = 'https://raw.githubusercontent.com/MaestriaCienciasCognitivas/ncc/main/book/static/Cuaderno7_Emodat.mat'
response = requests.get(url)
response.raise_for_status()
ecog_data = sp.io.loadmat(io.BytesIO(response.content))
display(ecog_data.keys())

In [ ]:
data =  np.squeeze(ecog_data['data'])

# Tasa de muestreo, número de muestras por segundo
srate = int(ecog_data['srate'].item())

print(f"Tasa de muestreo: {np.round(srate, 2)}")
print(f"Número de muestras: {len(data)}")
print(f"Segundos registrados: {np.round(len(data) / srate)}")

# Graficamos los datos en el primer segundo del registro
plt.plot(np.arange(0, srate) / srate, data[0:int(srate)])
plt.ylabel('Voltage (uV)')
plt.xlabel('Time [s]')
plt.title('ECoG Signal in Time Domain')
plt.show()

La tasa de muestreo es importante para cómo analizamos los datos en el dominio de la frecuencia. Cuando grabamos una señal de forma digital, la medimos a una cierta tasa. Por esta razón, no conocemos la amplitud exacta de una señal digital en cada punto en el tiempo. Nuestra mejor aproximación es la muestra más cercana a un punto dado en el tiempo.

Si tenemos una tasa de muestreo más alta, podemos obtener una mejor **resolución temporal**, pero almacenar demasiadas muestras puede hacer que el **tamaño del archivo** sea demasiado grande y consuma demasiada **memoria**.

En este caso, levantamos $1017,25$ muestras de datos por segundo. Si lo deseamos, podemos remuestrear los datos usando una tasa de muestreo diferente.

El remuestreo, la Densidad Espectral de Potencia (PSD) y muchas otras cosas se estudian en Procesamiento Digital de Señales. Puedes leer más sobre estos temas aquí: http://dspguide.com/.

En la siguiente celda, vamos a remuestrear nuestros datos a una tasa de muestreo diferente. Alguien podría **reducir** la tasa de muestreo para hacer que el archivo sea más pequeño, pero nosotros vamos a remuestrear porque nos ayudará a tener un número par de muestras por segundo más adelante.

In [ ]:
new_srate = 1024
data_resampled = sp.signal.resample(data, int(np.floor(len(data)*(new_srate/srate))))

print(f"Tasa de muestreo: {new_srate}")
print(f"Número de muestras: {len(data_resampled)}")
print(f"Segundos registrados: {np.round(len(data_resampled) / new_srate)}")

# Graficamos los datos en el primer segundo del registro
plt.plot(np.arange(0, new_srate) / new_srate, data_resampled[0:new_srate])
plt.ylabel('Voltaje (uV)')
plt.xlabel('Tiempo (s)')
plt.title('Señal ECoG en el dominio temporal')
plt.show()

*Nota: Compare las dos gráficas, ¿son iguales?*

## Transformadas de Fourier

Hay una función matemática bastante compleja llamada transformada de Fourier.

Esta función mide la amplitud de cada frecuencia presente en una señal. Las matemáticas necesarias para hacerlo son bastante complicadas, pero no necesitamos meternos demasiado en eso.

Imaginá que tocás un acorde en un piano. Si aplicás una transformada de Fourier a la señal de sonido, podrías ver con qué fuerza presionaste cada una de las teclas.

En el cerebro, hay muchas frecuencias presentes en una misma señal, y no son tan simples como un acorde con tres notas discretas.

En las próximas celdas voy a crear una señal compleja y mostrarte cómo ejecutar una transformada de Fourier sobre ella.

In [ ]:
f = 1024 #sampling frequency
dur = 10 #10 seconds of signal
freq = 7 #7 Hz signal
freq2 = 130 #130 Hz signal
t = np.arange(0, dur, 1/f) #times for d ### gneraa un vector en el tiempo iniciando cero , genera dos señales señasl 1 es
sig1 = np.sin(2 * np.pi * freq * t) #10 Hz wavelength
sig1 = 1.5 * sig1 #increase the power of signal  ### se las puede agregar a quitar ruido  cambiando el sin1¨22
sig2 = np.sin(2 * np.pi * freq2 * t) #130 Hz wavelength

plt.plot(t[0:512],sig1[0:512]+sig2[0:512], label = 'complex signal') #plot 0.5 seconds of data
plt.show()

Qué tal si cambiamos las dos frecuencias y generamos una onda con componentes en alpha (10 Hz) y gamma alto (40 Hz)? Probemos, creen la señal y grafíquenla nuevamente.


In [ ]:
# incluir acá el código para generar una nueva señal con dos nuevas frecuencias (10 y 40)

Para construir la señal del gráfico de arriba, sumé dos ondas sinusoidales.

Hay una onda más lenta con gran amplitud y otra más rápida con una amplitud menor. Asegurate de identificarlas.

Podemos usar una transformada de Fourier para demostrar que hay dos señales que, al sumarlas, producen esta señal y para mostrar cuáles son sus frecuencias.

In [ ]:
# calculating fourier transform of complex signal
fourier = np.fft.fft(sig1+sig2, 1024) #muestra los cosenos y los pesos

# plotting up to 150 Hz
plt.plot(abs(fourier[0:150]))
plt.ylabel('Power')
plt.xlabel('Frequency (Hz)')
plt.title('FFT of a complex signal')
plt.show()

# la salida que da es una parte real y la parte imaginada de la función
# el vector de la transformada fsx_bins te dice a que frecuencia se realizan esos arange
# en la gráfica lo que te muestra son las frecuencias más importantes de la potencia
# intensidad equipara la amplitud en el lenguaje
# y la potencia en el lenguaje
display(fourier)

En este gráfico estamos viendo la potencia de todas las frecuencias presentes en nuestra señal.

Podemos ver claramente un pico en 7 Hz y otro en 130 Hz.  
También se nota que el pico de 7 Hz tiene una amplitud mayor que el de 130 Hz.

Bastante impresionante.

Sin embargo, al hacer esto no vemos información sobre el momento temporal de la señal. Cuando observamos una señal en el espacio de frecuencias, no podemos saber nada sobre el **desfase de las oscilaciones**.

Aun así, la información de fase sigue existiendo: está almacenada en números complejos que no estamos graficando.  
Se llama una fourier **transformada** porque **no se pierde información al aplicarla**. Podés calcular la transformada de Fourier o la inversa una y otra vez sin perder datos.

Muy interesante, pero ya es demasiada matemática por hoy.

Cambiemos de tema hacia algo más atractivo: ¿cómo se ven una señal neuronal y su fft?

In [ ]:
plt.plot(np.arange(0, 1024) / 1024, data_resampled[0:1024]) #this is what about 1 second of EEG data looks like
plt.ylabel('Voltage (uV)')
plt.xlabel('Time [s]')
plt.title('ECoG Signal in Time Domain')
plt.show()

Arriba hay 1 segundo de datos neuronales. Podrías notar subidas y bajadas y concluir que es oscilatorio, pero no aporta mucho más. Ahora miremos su transformada de Fourier.

In [ ]:
fourier = np.fft.fft(data_resampled, 1024)

plt.plot(abs(fourier[0:150]))
plt.ylabel('Power')
plt.xlabel('Frequency (Hz)')
plt.title('FFT of ECoG signal')
plt.show()

Guau. Está pasando mucho. Desglosemos esto:

Esta señal es muy ruidosa. Parece haber muchas longitudes de onda que contribuyen a ella. 

Las longitudes de onda más largas parecen tener una potencia mayor que las más cortas.

### Nota opcional

En realidad, la ecuación de esta línea es $P = 1/f$.

Esta es una proporción matemática muy especial llamada **distribución de ley de potencia** o **ruido rosa**.  

Las relaciones 1/f aparecen por todos lados en la naturaleza de formas sorprendentes: el tamaño de los terremotos, los movimientos del mercado de valores, el tamaño de los cráteres en la Luna e incluso el espectro de potencia de la música popular siguen distribuciones de ley de potencia. Fenómenos realmente interesantes y misteriosos.

## Método de Welch

Volviendo a la neurociencia.
Hay una manera de suavizar nuestra fft sin perder demasiada información.  

La forma más común de hacerlo es usando el **método de Welch**.  

Básicamente, calculamos la fft de la señal en varias ventanas deslizantes y luego obtenemos el promedio del PSD de todas esas ventanas.

In [ ]:
f,pspec = sp.signal.welch(data_resampled, fs=new_srate, window='hann', nperseg=2*new_srate, noverlap=new_srate/2, nfft=None, detrend='linear', return_onesided=True, scaling='density')
#Try to figure out what the paramaters above are doing. What happens if you change them?

# Any frequencies with >150 Hz are going to be noise in ECoG data.
plt.plot(f[0:150],np.log(pspec[0:150]))
plt.ylabel('Power')
plt.xlabel('Frequency (Hz)')
plt.title("Welch's PSD of ECoG signal")
plt.show()

¡Mirá eso!

Ahora la señal es mucho más limpia y parecen aparecer algunos picos interesantes alrededor de 8, 14 y 25 Hz.  

Estos se conocen como las bandas **theta**, **alfa** y **beta**.  

Estas longitudes de onda se estudian desde la década de 1950, pero en realidad todavía sabemos poco sobre ellas.  

<http://en.wikipedia.org/wiki/Electroencephalography>